In [1]:
from nlp import load_dataset, Dataset
import pandas as pd
import re
from transformers import DistilBertTokenizer
from transformers import Trainer, TrainingArguments, DistilBertForSequenceClassification
from datasets import load_metric
import numpy as np
from torch.nn import Softmax
from torch.utils.data import DataLoader
from torch import argmax


In [2]:
tweets = pd.read_csv('../data/tweets_sample.csv', encoding='ISO-8859-1')

tweets['date'] = pd.to_datetime(tweets['TweetAt']).astype('datetime64[ms]')

tweets.index = tweets['date']
del tweets['date']

tweets.sort_index(inplace=True)

tweets.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,month
date,,,,,,,
2020-01-04,29309,74261,"Tidal County, Knoxville, TN",01-04-2020,Overwhelmed with info Before you act stop Ask ...,Positive,1
2020-01-04,29313,74265,NaN,01-04-2020,Webinar 4 2 3 30p ET Navigating Today s Market...,Neutral,1
2020-01-04,29314,74266,Edmonton,01-04-2020,The provincial government is increasing its fo...,Negative,1
2020-01-04,29315,74267,News & Bible verses USA,01-04-2020,Coronavirus: Oil prices could turn negative as...,Negative,1
2020-01-04,29316,74268,"Fort George G. Meade, Md.",01-04-2020,Malwarebytes rounded up some useful tips for s...,Extremely Positive,1


In [ ]:
# def aum_validation(
#             self, embedded_model: EmbeddedTrainingPhrases,
#             embedded_inputs: EmbeddedTrainingPhrases) -> List[db.AumOutcome]:
combined_embeddings = np.concatenate(
    (embedded_model.embedding_matrix, embedded_inputs.embedding_matrix), axis=0)
intents = list(set(embedded_model.intents))
intent_to_class = {intent: i for i, intent in enumerate(intents)}
mislabelled_class = len(intents)
examples_to_reassign = set()
number_to_reassign = ceil(len(embedded_model.embedding_matrix) / (len(intents) + 1))
while len(examples_to_reassign) < number_to_reassign:
    examples_to_reassign.add(randint(0, len(embedded_model.training_phrases)))
threshold_embeddings, target_model_classes = [], np.ndarray(((len(embedded_model.intents)),), dtype=int)
for i, intent in enumerate(embedded_model.intents):
    intent_class = intent_to_class[intent]
    if i in examples_to_reassign:
        target_model_classes[i] = mislabelled_class
        threshold_embeddings.append(embedded_model.embedding_matrix[i, :])
        continue
    target_model_classes[i] = intent_class

threshold_embedding_matrix = np.stack(threshold_embeddings, axis=0)
target_input_classes = np.asarray([intent_to_class[intent] for intent in embedded_inputs.intents])
combined_target_classes = np.concatenate((target_model_classes, target_input_classes), axis=0)

aum_validation_callback = AreaUnderMargin(
    embedded_inputs.embedding_matrix, target_input_classes, sparse=True)
aum_threshold_callback = AreaUnderMargin(
    threshold_embedding_matrix, [mislabelled_class] * len(threshold_embedding_matrix), sparse=True)
model = Classifier(hidden_layer_units=[100], num_classes=len(intents) + 1)
model.train(
    combined_embeddings, combined_target_classes, sparse=True, epochs=50,
    callbacks=[aum_validation_callback, aum_threshold_callback])
threshold = np.percentile(aum_threshold_callback.avg_aum, 99)
outcomes = []
for i, training_phrase in enumerate(embedded_inputs.training_phrases):
    outcomes.append(
        db.AumOutcome(
            labelled_intent_name=embedded_inputs.intents[i].name,
            external_intent_id=embedded_inputs.intents[i].id,
            training_phrase=training_phrase,
            area_under_margin=aum_validation_callback.avg_aum[i],
            margin_threshold=threshold
        )
    )
outcomes